In [74]:
from qutip import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import scipy.integrate as sci
import scipy.special as scs





N=3
n_oscillators=1
gamma=0.0
gammam=1
delta=20*gammam
eta=100*gammam
n=2
theta=np.pi/2
m=4
tmax=250
samples=300
tau=100
eigen=5


integrate=lambda x,n,m:np.sqrt(2.0/np.pi)*np.exp(-2*x**2)*scs.eval_hermite(n,np.sqrt(2)*x)*scs.eval_hermite(m,np.sqrt(2)*x)/np.sqrt(2**(n+m)*scs.gamma(n+1)*scs.gamma(m+1))
        
    
M1=np.zeros((N,N))


for i in range(N):
    for j in range(N):
        M1[i,j]=sci.quad(integrate,0.0,np.inf,args=(i,j),limit=50000)[0]
M1=Qobj(M1)


f=lambda t:np.abs(eta)*np.tanh(t/tau)

a=destroy(N)
ad=create(N)



psi0=basis(N,0)*np.random.uniform(0.0,1.0)

for i in range(1,N):
    psi0+=basis(N,i)*np.random.uniform(0.0,1.0)

psi0=psi0.unit()




rho0=ket2dm(psi0)

tlist=np.linspace(0.0,1500,1200)

H0=delta*ad*a
Ht=(a**n*np.exp(theta*n*1.0j)-ad**n*np.exp(-theta*n*1.0j))*1.0j

jump=[np.sqrt(gamma)*a,np.sqrt(gammam)*(a**m)]
ops=[a,ad*a,a*a,a**4,ad*a*ad*a,M1*a*M1,M1*a*a*M1]

In [ ]:
result=mesolve([H0,[Ht,f]],rho0,tlist,c_ops=jump,e_ops=ops,options={
    "store_final_state":True,"store_states":True,"nsteps":100000000,"progress_bar":"tqdm","normalize_output":False,"atol":1e-9,"rtol":1e-9
})

  0%|          | 0/1199 [00:00<?, ?it/s]

In [ ]:
filas=np.zeros((tlist.size,len(ops)),dtype=np.complex128)

for i in range(len(ops)):
    filas[:,i]=result.expect[i]

In [ ]:

auto=np.zeros((eigen,60),dtype=np.complex128)
cont=0
for i in range(0,tlist.size,20):
    L = liouvillian(H0+f(tlist[i])*Ht, jump)
    # Obtener el espectro del Liuvilliano
    auto[:,cont] = L.eigenenergies()[-eigen:]
    cont+=1

pd.DataFrame(auto,columns=tlist[0:-1:20]).to_csv("espectro_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(eta)+"_theta_"+str(theta)+".csv")

In [ ]:
psi0=ket2dm(basis(N,0))

In [ ]:
result=mesolve([H0,[Ht,f]],rho0,tlist,c_ops=jump,e_ops=ops,options={
    "store_final_state":True,"store_states":True,"nsteps":100000000,"progress_bar":"tqdm","normalize_output":False,"atol":1e-9,"rtol":1e-9
})

In [ ]:
filas=np.zeros((tlist.size,len(ops)),dtype=np.complex128)

for i in range(len(ops)):
    filas[:,i]=result.expect[i]

resultados=pd.DataFrame(filas,columns=["a","N","a*a","a**4","ad*a*ad*a","MaM","M*a*a*M"])

resultados.to_csv("valores_esperados_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(eta)+"_theta_"+str(theta)+".csv")

In [ ]:
psi0=ket2dm(coherent(N,eta**(1/6)))

result=mesolve([H0,[Ht,f]],rho0,tlist,c_ops=jump,e_ops=ops,options={
    "store_final_state":True,"store_states":True,"nsteps":100000000,"progress_bar":"tqdm","normalize_output":False,"atol":1e-9,"rtol":1e-9
})

In [ ]:
filas=np.zeros((tlist.size,len(ops)),dtype=np.complex128)

for i in range(len(ops)):
    filas[:,i]=result.expect[i]

resultados=pd.DataFrame(filas,columns=["a","N","a*a","a**4","ad*a*ad*a","MaM","M*a*a*M"])

resultados.to_csv("valores_esperados_lobulo_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(eta)+"_theta_"+str(theta)+".csv")